<a href="https://colab.research.google.com/github/financieras/math_for_ai/blob/main/articulos/Dominando_Pandas_GroupBy_Parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()  # Sube el archivo kaggle.json (descárgalo desde Kaggle > Cuenta > API > Create New API Token)

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"valoro","key":"08ff511228f99c4b209ce742ea557c84"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c titanic  # Descarga el ZIP del dataset Titanic

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 98.6MB/s]


In [5]:
!unzip titanic.zip  # Extrae train.csv, test.csv y gender_submission.csv

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [6]:
# Importar librerías
import pandas as pd
import numpy as np

# Hacer que la salida se vea bien
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', None)

In [7]:
# Cargar dataset del Titanic
df = pd.read_csv('train.csv')

# Verificación rápida
print(f"Dataset shape: {df.shape}")
df.head(3)

Dataset shape: (891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S


---

## **3. `.agg()` - Múltiples Agregaciones a la Vez**

Este es **EL** método más útil que aprenderás hoy. En el análisis de datos del mundo real, rara vez quieres solo una estadística; quieres media, mediana, desviación estándar, mínimo, máximo, conteo, todo a la vez.

### **3.1. Sintaxis Básica: Múltiples Funciones en Una Columna**

En lugar de llamar a `.mean()`, luego `.median()`, luego `.std()` por separado, hazlo todo de una vez:

In [8]:
# Múltiples estadísticas para Age por Sex
df.groupby('Sex')['Age'].agg(['mean', 'median', 'std', 'min', 'max', 'count'])

,mean,median,std,min,max,count
Sex,,,,,,
female,27.92,27.00,14.11,0.75,63.00,261
male,30.73,29.00,14.68,0.42,80.00,453


**¡Perfecto!** Seis estadísticas en una línea. Esto es lo que usarás constantemente en proyectos reales.

Observa:
- Pasamos una **lista** de nombres de funciones como cadenas
- Cada función se convierte en una columna en el resultado
- El resultado está limpio y listo para usar

### **3.2. Funciones Diferentes para Columnas Diferentes**

¿Qué pasa si quieres estadísticas diferentes para columnas diferentes? Fácil:

In [9]:
# Diferentes agregaciones para Age y Fare
df.groupby('Pclass').agg({
    'Age': ['mean', 'median', 'count'],
    'Fare': ['mean', 'max', 'min']
})

Age               Fare            
        mean median count  mean    max  min
Pclass                                     
1      38.23  37.00   186 84.15 512.33 0.00
2      29.88  29.00   173 20.66  73.50 0.00
3      25.14  24.00   355 13.68  69.55 0.00

¡Esto es poderoso! Observa la sintaxis:
- Pasamos un **diccionario** a `.agg()`
- Claves = nombres de columnas
- Valores = lista de funciones

Esto crea un reporte con exactamente las estadísticas que necesitas para cada variable.

### **3.3. Renombrar Columnas para Mayor Claridad**

Los nombres de columnas por defecto son desordenados (Age_mean, Age_median, etc.). Arreglémoslo:

In [10]:
# Agregaciones con nombre (Pandas 0.25+)
result = df.groupby('Pclass').agg(
    Mean_age=('Age', 'mean'),
    Median_age=('Age', 'median'),
    Passengers=('Age', 'count'),
    Mean_fare=('Fare', 'mean'),
    Max_fare=('Fare', 'max')
).round(2)

result

,Mean_age,Median_age,Passengers,Mean_fare,Max_fare
Pclass,,,,,
1,38.23,37.00,186,84.15,512.33
2,29.88,29.00,173,20.66,73.50
3,25.14,24.00,355,13.68,69.55


**¡Perfecto!** Nombres de columnas limpios y profesionales. Así es como deberías escribir código de producción.

La sintaxis: `Nuevo_nombre=(columna, funcion)`

### **3.4. Usando Funciones Personalizadas con `.agg()`**

También puedes pasar tus propias funciones:

In [11]:
# Función personalizada: calcular rango (max - min)
def age_range(series):
    return series.max() - series.min()

df.groupby('Sex').agg(
    Mean_age=('Age', 'mean'),
    Age_range=('Age', age_range),
    Total_fare=('Fare', 'sum')
).round(2)

,Mean_age,Age_range,Total_fare
Sex,,,
female,27.92,62.25,13966.66
male,30.73,79.58,14727.29


**Consejo profesional:** Puedes mezclar funciones integradas (como `'mean'`) con funciones personalizadas (como `age_range`).

### **3.5. Ejemplo del Mundo Real: Reporte Estadístico Completo**

Creemos un reporte profesional combinando todo:

In [12]:
# Reporte estadístico completo por clase de pasajero
report = df.groupby('Pclass').agg(
    Total_passengers=('PassengerId', 'count'),
    Survivors=('Survived', 'sum'),
    Survival_rate=('Survived', lambda x: f"{x.mean()*100:.1f}%"),  # Formatear como porcentaje
    Avg_age=('Age', 'mean'),
    Median_age=('Age', 'median'),
    Avg_fare=('Fare', 'mean'),
    Max_fare=('Fare', 'max')
)

# Formatear columnas numéricas
report['Avg_age'] = report['Avg_age'].round(1)
report['Median_age'] = report['Median_age'].round(1)
report['Avg_fare'] = report['Avg_fare'].round(2)
report['Max_fare'] = report['Max_fare'].round(2)

report

,Total_passengers,Survivors,Survival_rate,Avg_age,Median_age,Avg_fare,Max_fare
Pclass,,,,,,,
1,216,136,63.0%,38.20,37.00,84.15,512.33
2,184,87,47.3%,29.90,29.00,20.66,73.50
3,491,119,24.2%,25.10,24.00,13.68,69.55


**Esto es lo que harás en proyectos reales:** crear reportes completos en 10 líneas de código.

---